*13.1 데이터 API*

In [ ]:

import tensorflow as tf
X = tf.range(10) #0부터 9까지의 10개의 아이템을 가짐
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

tf.data.Dataset.from_tensor_slices() 사용해 전체 데이터셋 생성

In [ ]:
for item in dataset:
  print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)


데이터셋이 준비되었습니다!

*13.1.1 연쇄변환*

In [ ]:
dataset = dataset.repeat(3). batch(7)
for item in dataset:
  print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)


In [ ]:
dataset = dataset.repeat(3).batch(7, drop_remainer=True)
for item in dataset:
  print(item)

TypeError: ignored

책에서 batch()메서드를 drop_remainder = True로 호출하면 모든 배치가 일정한 크기로 맞춰진다고 했는데 실패했어요 🥲

데이터셋 메서드는 데이터셋을 바꾸지 않고 새로운 데이터셋을 만드므로 꼭 새로운 데이터셋을 반환받아야 합니다.

In [ ]:
dataset = dataset.map(lambda x:x*2) #아이템:[0,2,4,6,8,10,12]



* 모든 아이템에 2를 곱해 새로운 데이터셋을 만듭니다.
*   항목 추가



In [ ]:
dataset = dataset.apply(tf.data.experimental.unbatch())

apply() 메서드는 unbatch() 함수를 적용하는데 현재 실험적입니다.

새로 만들어진 데이터셋의 각 아이템은 7개의 정수로 이루어진 배치가 아니라 하나의 정수텐서가 됩니다.

하지만 텐서플로에서는 더이상 쓰이지 않는다고 되어 있었습니다.

In [ ]:
dataset = dataset.filter(lambda x: x<10)


In [ ]:
for item in dataset.take(3):
  print(item)


*   take()메서드

*   데이터셋에 있는 몇 개의 아이템만 필터링해 볼 수도 있습니다.



*13.1.2 데이터 셔플링*



*   원본 -> 버퍼 -> 반환
*   원본 데이터셋의 모든 아이템을 쓸 때까지
*   버퍼가 비워질 때까지 (버퍼 충분히 크게)
*   메모리 크기 초과 X
*   데이터셋 크기 초과 X


---


[링크 텍스트](https://)






In [ ]:
dataset = tf. data. Dataset. range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)
for item in dataset:
  print(item)

정수 0에서 9까지 세 번 반복된 데이터셋을 버퍼 크기5, 랜덤시드 42 통해 셔플링한 것.

In [ ]:
dataset = tf. data. Dataset. range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5, seed=42, reshuffle_each_iteration=False).batch(7)
for item in dataset:
  print(item)

반복마다 동일한 순서를 사용해야 한다면 reshuffle_each_interaction=False를 지정해야 하는데 

동일한 순서가 나오지 않았어요 🥲

메모리 용량보다 큰 데이터셋은 셔플링만을 충분하지 않아

원본 데이터 자체를 섞어야 합니다.

ex. 리눅스에서 shuf 명령어 통해 텍스트 섞음

에포크마다 한 번 더 섞기!

편향 방지

*13.1.3 데이터 전처리*

In [ ]:
X_mean, X_std = [0,1]
n_inputs = 8

def preprocess(line):
  defs =[0.] * n_inputs + [tf.constant([],dtype=tf.float32)]
  fields = tf.io.decode_csv(line,record_defaults=defs)
  x= tf.stack(fields[:-1])
  y= tf.stack(fields[-1:])
  return (x - X_mean) / X_std, y

X_mean과 X_std는 1개씩 8개의 실수를 가진 1D 텐서

파싱할 라인과 CSV파일의 각 열에 대한 기본 값

decode_csv()   스칼라 텐서 리스트 반환

tf.stack 함수 (마지막 열 빼고) 스칼라 텐서 -> 하나의 값을 가진 1D 텐서





In [ ]:
preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')

(<tf.Tensor: shape=(8,), dtype=float32, numpy=
 array([   4.2083,   44.    ,    5.3232,    0.9171,  846.    ,    2.337 ,
          37.47  , -122.2   ], dtype=float32)>,
 <tf.Tensor: shape=(1,), dtype=float32, numpy=array([2.782], dtype=float32)>)

In [ ]:
from keras.datasets import mnist
(X_train, Y_class_train),(X_train, Y_class_test) = mnist.load_data()

번외

케라스를 이용해 mnist 손글씨

*13.1.4 데이터 적재와 전처리를 합치기*

In [ ]:
def csv_reader_dataset(filepaths,repeat=1, n_readers=5,
                       n_read_threads=None, shuffle_buffer_size= 10000,
                       n_parse_threads=5, batch_size = 32):
  dataset = tf.data.Dataset.list_files(filepaths).repeat(repeat)
  dataset = dataset.interleave(
      lambda filepath: tf.data.TextLineDataser(filepath).skip(1),
      cycle_length=n_readers, num_parallel_calls=n_read_threads)
  dataset = dataset.shuffle(shuffle_buffer_size)
  dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
  return dataset.batch(batch_size).prefetch(1)
  

CSV 파일에서 캘리포니아 주택 데이터셋 적재, 전처리, 셔플링, 반복, 배치를 적용한 데이터셋

*13.1.5 프리페치*



*   prefetch(1)
*   항상 한 배치가 미리 준비되도록
*   성능 향상: interleave()와 map() 메서드 호출할 때 num_parallel_calls 매개변수 지정
*   CPU와 GPU를 동시에 사용: GPU가 한 배치 처리할 때 CPU가 그 다음 배치 준비






자주 쓰는 데이터셋 메서드
*   concatenate() 
*   zip()
*   window()
*   reduce()
*   shard()
*   flat_map()
*   padded_batch()








*13.1.6 tf.keras와 데이터셋 이용하기*

keras에서 반복을 처리하므로 반복을 지정할 필요가 없다.

데이터셋(검증 세트, 테스트 세트) -> 케라스 모델 만들기 -> 훈련셋과 검증셋 전달